In [ ]:
import numpy as np
import pandas as pd
from firebase_admin import firestore, initialize_app

initialize_app()
database = firestore.client()
col = database.collection(u'heartfelt_data')

In [ ]:
from gcp_utils import constants

col.add(constants.RAW_VALID_SAMPLE)

In [ ]:
data = [x.to_dict() for x in col.where(u'sample_id', u'==', u'123456789').stream()][0]

In [ ]:
pd.Series(data['abp']).plot()